In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 路徑
load_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/rssi/combined_csi_rssi_slim2.4G.csv"

# 讀取資料
df = pd.read_csv(load_path)
print("✅ 讀取成功，資料維度：", df.shape)

# 只用CSI amplitude (前48欄)
amp = df.iloc[:, 0:48].values   # shape: (N, 48)
labels = df["Label"].values     # shape: (N,)
orig_idx = df.index.values      # 原始index方便對照

# train/val/test切分
amp_train, amp_temp, y_train, y_temp, idx_train, idx_temp = train_test_split(
    amp, labels, orig_idx, test_size=0.3, random_state=42
)
amp_val, amp_test, y_val, y_test, idx_val, idx_test = train_test_split(
    amp_temp, y_temp, idx_temp, test_size=1/3, random_state=42
)

print(f"Train: {amp_train.shape}, Val: {amp_val.shape}, Test: {amp_test.shape}")


✅ 讀取成功，資料維度： (19649, 53)
Train: (13754, 48), Val: (3930, 48), Test: (1965, 48)


In [38]:
# 訓練用
train_data = amp_train    # shape: [N_train, 48]
train_labels = y_train    # shape: [N_train]

# 驗證用
val_data = amp_val        # shape: [N_val, 48]
val_labels = y_val        # shape: [N_val]

# 測試用
test_data = amp_test      # shape: [N_test, 48]
test_labels = y_test      # shape: [N_test]


In [39]:
# 假設你已經定義好這兩個class
class SiameseSubNet1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(48, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class SiameseNetwork1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.subnet = SiameseSubNet1D()
    def forward(self, x1, x2):
        out1 = self.subnet(x1)
        out2 = self.subnet(x2)
        distance = F.pairwise_distance(out1, out2)
        return distance

# 重點：這裡要用 SiameseNetwork1D
model = SiameseNetwork1D().cuda()


In [40]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=4.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output, label):
        # label: 1=positive, 0=negative
        loss = 0.5 * label * torch.pow(output, 2) + \
               0.5 * (1 - label) * torch.pow(torch.clamp(self.margin - output, min=0.0), 2)
        return loss.mean()



In [41]:
import torch
import numpy as np
from torch.utils.data import Dataset

class CSIAllPairsDataset(Dataset):
    def __init__(self, X, y, T1):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = np.array(y)
        self.label_set = np.unique(self.y)
        self.T1 = T1
        self.idx_by_label = {label: np.where(self.y == label)[0][:T1] for label in self.label_set}

        self.fingerprints = {}  # 每個RP一個fingerprint
        for label in self.label_set:
            idxs = self.idx_by_label[label]
            self.fingerprints[label] = self.X[idxs].mean(dim=0)  # 平均後 shape: [48]

        # 所有pair列表：(data_idx, fp_label, label)
        # 正樣本: label=1, 負樣本: label=0
        self.pairs = []
        for rp in self.label_set:
            idxs = self.idx_by_label[rp]
            for i in idxs:
                for fp_rp in self.label_set:
                    label = 1 if rp == fp_rp else 0
                    self.pairs.append((i, fp_rp, label))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        i, fp_label, label = self.pairs[idx]
        x1 = self.X[i]
        x2 = self.fingerprints[fp_label]
        return x1, x2, torch.tensor(label, dtype=torch.float32)



In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def eval_siamese_acc(model, data_loader, margin=4.0):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x1, x2, label in data_loader:
            x1, x2, label = x1.to(device), x2.to(device), label.to(device)
            distance = model(x1, x2)
            pred = (distance < margin).float()  # 距離小於 margin 視為同類
            correct += (pred == label).sum().item()
            total += label.size(0)
    return correct / total

def compute_mean_distance_error(y_true, y_pred, coordinates):
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label not in coordinates or pred_label not in coordinates:
            print(f"Warning: Label {true_label} or {pred_label} not in coordinates, skipping.")
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        errors.append(np.linalg.norm(pred_coord - true_coord))
    return np.mean(errors), errors



COORDINATES = {
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import os
import copy

# ==== 1. 模型定義 ====

class SiameseSubNet1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(48, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class SiameseNetwork1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.subnet = SiameseSubNet1D()
    def forward(self, x1, x2):
        out1 = self.subnet(x1)
        out2 = self.subnet(x2)
        distance = F.pairwise_distance(out1, out2)
        return distance

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=4.0):
        super().__init__()
        self.margin = margin
    def forward(self, output, label):
        loss = 0.5 * label * torch.pow(output, 2) + \
               0.5 * (1 - label) * torch.pow(torch.clamp(self.margin - output, min=0.0), 2)
        return loss.mean()

# ==== 2. Dataset 全pair實作 ====

class CSIAllPairsDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, T1):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = np.array(y)
        self.label_set = np.unique(self.y)
        self.T1 = T1
        self.idx_by_label = {label: np.where(self.y == label)[0][:T1] for label in self.label_set}
        self.fingerprints = {}
        for label in self.label_set:
            idxs = self.idx_by_label[label]
            self.fingerprints[label] = self.X[idxs].mean(dim=0)
        self.pairs = []
        for rp in self.label_set:
            idxs = self.idx_by_label[rp]
            for i in idxs:
                for fp_rp in self.label_set:
                    label = 1 if rp == fp_rp else 0
                    self.pairs.append((i, fp_rp, label))
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        i, fp_label, label = self.pairs[idx]
        x1 = self.X[i]
        x2 = self.fingerprints[fp_label]
        return x1, x2, torch.tensor(label, dtype=torch.float32)

# ==== 3. 評估函數 ====

def eval_siamese_acc(model, data_loader, margin=4.0):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x1, x2, label in data_loader:
            x1, x2, label = x1.to(device), x2.to(device), label.to(device)
            distance = model(x1, x2)
            pred = (distance < margin).float()
            correct += (pred == label).sum().item()
            total += label.size(0)
    return correct / total

def compute_mean_distance_error(y_true, y_pred, coordinates):
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label not in coordinates or pred_label not in coordinates:
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        errors.append(np.linalg.norm(pred_coord - true_coord))
    return np.mean(errors), errors

# ==== 4. 訓練參數與資料準備 ====

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_runs = 5
epochs = 200
patience = 20
T1 = 7
margin = 4.0


train_dataset = CSIAllPairsDataset(amp_train, y_train, T1)
val_dataset = CSIAllPairsDataset(amp_val, y_val, T1)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

test_accs, test_mdes, all_run_errors = [], [], []
os.makedirs("siamese_results", exist_ok=True)

for run in range(1, num_runs + 1):
    print(f"\n=== Run {run}/{num_runs} ===")

    model = SiameseNetwork1D().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=0.1, rho=0.95, eps=1e-8)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=15, min_lr=1e-6, verbose=True)
    criterion = ContrastiveLoss(margin=4.0)

    best_val_loss = float('inf')
    counter = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for x1, x2, label in train_loader:
            x1, x2, label = x1.to(device), x2.to(device), label.to(device)
            optimizer.zero_grad()
            distance = model(x1, x2)
            loss = criterion(distance, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)

        # === 驗證（以 contrastive loss 作 early stop/lr decay依據） ===
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x1, x2, label in val_loader:
                x1, x2, label = x1.to(device), x2.to(device), label.to(device)
                distance = model(x1, x2)
                val_loss += criterion(distance, label).item() * x1.size(0)
        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)
        print(f"Epoch [{epoch+1}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.5f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            best_state = copy.deepcopy(model.state_dict())
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # ==== 測試階段：fingerprint-based 位置預測與 MDE ====
    model.load_state_dict(best_state)
    model.eval()
    # 準備 fingerprint
    fingerprints = []
    rp_list = np.unique(y_train)
    for rp in rp_list:
        idxs = np.where(y_train == rp)[0][:T1]
        fp = torch.tensor(amp_train[idxs].mean(axis=0), dtype=torch.float32).to(device)
        fingerprints.append(fp)
    fingerprints = torch.stack(fingerprints)

    all_true, all_pred = [], []
    for i in range(len(amp_test)):
        test_csi = torch.tensor(amp_test[i], dtype=torch.float32).unsqueeze(0).to(device)
        min_dist = float('inf')
        pred_rp = None
        with torch.no_grad():
            for j, fp in enumerate(fingerprints):
                dist = model.subnet(test_csi).sub(model.subnet(fp.unsqueeze(0))).norm(p=2).item()
                if dist < min_dist:
                    min_dist = dist
                    pred_rp = rp_list[j]
        all_true.append(y_test[i])
        all_pred.append(pred_rp)
    y_true = np.array(all_true)
    y_pred = np.array(all_pred)
    acc = 100 * np.mean(y_true == y_pred)
    mde, errors = compute_mean_distance_error(y_true, y_pred, COORDINATES)
    all_run_errors.append(errors)
    test_accs.append(acc)
    test_mdes.append(mde)
    print(f"✅ Run {run}: Acc = {acc:.2f}%, MDE = {mde:.4f}")

# ==== 儲存 summary 結果 ====
df = pd.DataFrame({
    "run": list(range(1, num_runs+1)),
    "accuracy": test_accs,
    "mde": test_mdes
})
df.to_csv("siamese_results/siamese_results.csv", index=False)

error_records = []
for run_idx, errors in enumerate(all_run_errors):
    for sample_idx, e in enumerate(errors):
        error_records.append({
            "run": run_idx + 1,
            "sample_idx": sample_idx + 1,
            "error": e
        })
df_errors = pd.DataFrame(error_records)
df_errors.to_csv("siamese_results/siamese_all_errors.csv", index=False)




=== Run 1/5 ===
Epoch [1], Train Loss: 29.7136, Val Loss: 17.1391, LR: 0.10000
Epoch [2], Train Loss: 12.4512, Val Loss: 9.3461, LR: 0.10000
Epoch [3], Train Loss: 6.8889, Val Loss: 6.1210, LR: 0.10000
Epoch [4], Train Loss: 4.7025, Val Loss: 4.6615, LR: 0.10000
Epoch [5], Train Loss: 3.5297, Val Loss: 3.7709, LR: 0.10000
Epoch [6], Train Loss: 2.8663, Val Loss: 3.1602, LR: 0.10000
Epoch [7], Train Loss: 2.3482, Val Loss: 2.7076, LR: 0.10000
Epoch [8], Train Loss: 1.9965, Val Loss: 2.3872, LR: 0.10000
Epoch [9], Train Loss: 1.6951, Val Loss: 2.0729, LR: 0.10000
Epoch [10], Train Loss: 1.4644, Val Loss: 1.8494, LR: 0.10000
Epoch [11], Train Loss: 1.2823, Val Loss: 1.6767, LR: 0.10000
Epoch [12], Train Loss: 1.1272, Val Loss: 1.5215, LR: 0.10000
Epoch [13], Train Loss: 1.0030, Val Loss: 1.4031, LR: 0.10000
Epoch [14], Train Loss: 0.8987, Val Loss: 1.3184, LR: 0.10000
Epoch [15], Train Loss: 0.8201, Val Loss: 1.2048, LR: 0.10000
Epoch [16], Train Loss: 0.7367, Val Loss: 1.1408, LR: 0.100

In [ ]:
# from torch.utils.data import DataLoader

# T1 = 7  # 或你想要的訓練CSI數，與dataset一致
# train_dataset = CSIAllPairsDataset(amp_train, y_train, T1)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # batch size可根據顯存調整

# model = SiameseNetwork1D().cuda()
# criterion = ContrastiveLoss(margin=4.0)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=0.001, rho=0.95, eps=1e-8)

# for epoch in range(200):
#     model.train()
#     running_loss = 0.0
#     for x1, x2, label in train_loader:
#         x1, x2, label = x1.cuda(), x2.cuda(), label.cuda()
#         optimizer.zero_grad()
#         distance = model(x1, x2)
#         loss = criterion(distance, label)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     print(f"Epoch [{epoch+1}], Loss: {running_loss/len(train_loader):.4f}")



Epoch [1], Loss: 54.6189
Epoch [2], Loss: 52.8977
Epoch [3], Loss: 52.1739
Epoch [4], Loss: 51.5112
Epoch [5], Loss: 51.8643
Epoch [6], Loss: 50.1378
Epoch [7], Loss: 49.5170
Epoch [8], Loss: 48.8729
Epoch [9], Loss: 48.2600
Epoch [10], Loss: 47.6342
Epoch [11], Loss: 47.0214
Epoch [12], Loss: 46.4497
Epoch [13], Loss: 45.8008
Epoch [14], Loss: 45.1735
Epoch [15], Loss: 44.6020
Epoch [16], Loss: 43.9874
Epoch [17], Loss: 43.3621
Epoch [18], Loss: 42.7914
Epoch [19], Loss: 42.2601
Epoch [20], Loss: 41.6988
Epoch [21], Loss: 41.1418
Epoch [22], Loss: 40.5986
Epoch [23], Loss: 40.0189
Epoch [24], Loss: 40.1967
Epoch [25], Loss: 38.9518
Epoch [26], Loss: 38.4030
Epoch [27], Loss: 37.8993
Epoch [28], Loss: 37.3822
Epoch [29], Loss: 36.9129
Epoch [30], Loss: 36.4061
Epoch [31], Loss: 35.9453
Epoch [32], Loss: 35.7501
Epoch [33], Loss: 34.9988
Epoch [34], Loss: 34.5065
Epoch [35], Loss: 34.0637
Epoch [36], Loss: 33.5875
Epoch [37], Loss: 33.1199
Epoch [38], Loss: 32.6558
Epoch [39], Loss: 32.

KeyboardInterrupt: 

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def eval_siamese_acc(model, data_loader, margin=4.0):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x1, x2, label in data_loader:
            x1, x2, label = x1.to(device), x2.to(device), label.to(device)
            distance = model(x1, x2)
            pred = (distance < margin).float()  # 距離小於 margin 視為同類
            correct += (pred == label).sum().item()
            total += label.size(0)
    return correct / total

def compute_mean_distance_error(y_true, y_pred, coordinates):
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label not in coordinates or pred_label not in coordinates:
            print(f"Warning: Label {true_label} or {pred_label} not in coordinates, skipping.")
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        errors.append(np.linalg.norm(pred_coord - true_coord))
    return np.mean(errors), errors



COORDINATES = {
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import os
import copy

# ==== 1. 模型定義 ====

class SiameseSubNet1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(48, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 64)
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


class SiameseNetwork1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.subnet = SiameseSubNet1D()
    def forward(self, x1, x2):
        out1 = self.subnet(x1)
        out2 = self.subnet(x2)
        distance = F.pairwise_distance(out1, out2)
        return distance

class ContrastiveLoss(nn.Module):
    def __init__(self, margin=4.0):
        super().__init__()
        self.margin = margin
    def forward(self, output, label):
        loss = 0.5 * label * torch.pow(output, 2) + \
               0.5 * (1 - label) * torch.pow(torch.clamp(self.margin - output, min=0.0), 2)
        return loss.mean()

# ==== 2. Dataset 全pair實作 ====

class CSIAllPairsDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, T1):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = np.array(y)
        self.label_set = np.unique(self.y)
        self.T1 = T1
        self.idx_by_label = {label: np.where(self.y == label)[0][:T1] for label in self.label_set}
        self.fingerprints = {}
        for label in self.label_set:
            idxs = self.idx_by_label[label]
            self.fingerprints[label] = self.X[idxs].mean(dim=0)
        self.pairs = []
        for rp in self.label_set:
            idxs = self.idx_by_label[rp]
            for i in idxs:
                for fp_rp in self.label_set:
                    label = 1 if rp == fp_rp else 0
                    self.pairs.append((i, fp_rp, label))
    def __len__(self):
        return len(self.pairs)
    def __getitem__(self, idx):
        i, fp_label, label = self.pairs[idx]
        x1 = self.X[i]
        x2 = self.fingerprints[fp_label]
        return x1, x2, torch.tensor(label, dtype=torch.float32)

# ==== 3. 評估函數 ====

def eval_siamese_acc(model, data_loader, margin=4.0):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x1, x2, label in data_loader:
            x1, x2, label = x1.to(device), x2.to(device), label.to(device)
            distance = model(x1, x2)
            pred = (distance < margin).float()
            correct += (pred == label).sum().item()
            total += label.size(0)
    return correct / total

def compute_mean_distance_error(y_true, y_pred, coordinates):
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label not in coordinates or pred_label not in coordinates:
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        errors.append(np.linalg.norm(pred_coord - true_coord))
    return np.mean(errors), errors

# ==== 4. 訓練參數與資料準備 ====

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_runs = 5
epochs = 200
patience = 20
T1 = 7
margin = 4.0


train_dataset = CSIAllPairsDataset(amp_train, y_train, T1)
val_dataset = CSIAllPairsDataset(amp_val, y_val, T1)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

test_accs, test_mdes, all_run_errors = [], [], []
os.makedirs("siamese_results", exist_ok=True)

for run in range(1, num_runs + 1):
    print(f"\n=== Run {run}/{num_runs} ===")

    model = SiameseNetwork1D().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=0.1, rho=0.95, eps=1e-8)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=15, min_lr=1e-6, verbose=True)
    criterion = ContrastiveLoss(margin=4.0)

    best_val_loss = float('inf')
    counter = 0
    best_state = None

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for x1, x2, label in train_loader:
            x1, x2, label = x1.to(device), x2.to(device), label.to(device)
            optimizer.zero_grad()
            distance = model(x1, x2)
            loss = criterion(distance, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)

        # === 驗證（以 contrastive loss 作 early stop/lr decay依據） ===
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for x1, x2, label in val_loader:
                x1, x2, label = x1.to(device), x2.to(device), label.to(device)
                distance = model(x1, x2)
                val_loss += criterion(distance, label).item() * x1.size(0)
        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)
        print(f"Epoch [{epoch+1}], Train Loss: {avg_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.5f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            best_state = copy.deepcopy(model.state_dict())
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # ==== 測試階段：fingerprint-based 位置預測與 MDE ====
    model.load_state_dict(best_state)
    model.eval()
    # 準備 fingerprint
    fingerprints = []
    rp_list = np.unique(y_train)
    for rp in rp_list:
        idxs = np.where(y_train == rp)[0][:T1]
        fp = torch.tensor(amp_train[idxs].mean(axis=0), dtype=torch.float32).to(device)
        fingerprints.append(fp)
    fingerprints = torch.stack(fingerprints)

    all_true, all_pred = [], []
    for i in range(len(amp_test)):
        test_csi = torch.tensor(amp_test[i], dtype=torch.float32).unsqueeze(0).to(device)
        min_dist = float('inf')
        pred_rp = None
        with torch.no_grad():
            for j, fp in enumerate(fingerprints):
                dist = model.subnet(test_csi).sub(model.subnet(fp.unsqueeze(0))).norm(p=2).item()
                if dist < min_dist:
                    min_dist = dist
                    pred_rp = rp_list[j]
        all_true.append(y_test[i])
        all_pred.append(pred_rp)
    y_true = np.array(all_true)
    y_pred = np.array(all_pred)
    acc = 100 * np.mean(y_true == y_pred)
    mde, errors = compute_mean_distance_error(y_true, y_pred, COORDINATES)
    all_run_errors.append(errors)
    test_accs.append(acc)
    test_mdes.append(mde)
    print(f"✅ Run {run}: Acc = {acc:.2f}%, MDE = {mde:.4f}")

# ==== 儲存 summary 結果 ====
df = pd.DataFrame({
    "run": list(range(1, num_runs+1)),
    "accuracy": test_accs,
    "mde": test_mdes
})
df.to_csv("siamese_results/siamese_results.csv", index=False)

error_records = []
for run_idx, errors in enumerate(all_run_errors):
    for sample_idx, e in enumerate(errors):
        error_records.append({
            "run": run_idx + 1,
            "sample_idx": sample_idx + 1,
            "error": e
        })
df_errors = pd.DataFrame(error_records)
df_errors.to_csv("siamese_results/siamese_all_errors.csv", index=False)




=== Run 1/5 ===


/home/mcs/anaconda3/envs/kyle_ai/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1], Train Loss: 0.2833, Val Loss: 0.7335, LR: 0.10000
Epoch [2], Train Loss: 0.2433, Val Loss: 0.5971, LR: 0.10000
Epoch [3], Train Loss: 0.2263, Val Loss: 0.5448, LR: 0.10000
Epoch [4], Train Loss: 0.2217, Val Loss: 0.5168, LR: 0.10000
Epoch [5], Train Loss: 0.2127, Val Loss: 0.5649, LR: 0.10000
Epoch [6], Train Loss: 0.2066, Val Loss: 0.4569, LR: 0.10000
Epoch [7], Train Loss: 0.2021, Val Loss: 0.4405, LR: 0.10000
Epoch [8], Train Loss: 0.2078, Val Loss: 0.5367, LR: 0.10000
Epoch [9], Train Loss: 0.1965, Val Loss: 0.4707, LR: 0.10000
Epoch [10], Train Loss: 0.1935, Val Loss: 0.4696, LR: 0.10000
Epoch [11], Train Loss: 0.1882, Val Loss: 0.4474, LR: 0.10000
Epoch [12], Train Loss: 0.1842, Val Loss: 0.4412, LR: 0.10000
Epoch [13], Train Loss: 0.1876, Val Loss: 0.4261, LR: 0.10000
Epoch [14], Train Loss: 0.1824, Val Loss: 0.4417, LR: 0.10000
Epoch [15], Train Loss: 0.1818, Val Loss: 0.4186, LR: 0.10000
Epoch [16], Train Loss: 0.1835, Val Loss: 0.4401, LR: 0.10000
Epoch [17], Train

KeyboardInterrupt: 